In [1]:
# 해당 파일은 협업 피터링으로 임베딩을 학습 후 컨텐츠 기반 필터리으로 유사도 기반 추천을 수행하는 추천 엔진

In [2]:
import numpy as np
from sklearn.decomposition import randomized_svd, non_negative_factorization
from surprise import Dataset

In [3]:
data = Dataset.load_builtin('ml-100k', prompt=False)
raw_data = np.array(data.raw_ratings, dtype=int)
# 0 부터 시작하기 위해(데이터는 1 부터 시작하기 때문)
raw_data[:,0] -= 1
raw_data[:,1] -= 1

In [4]:
n_users = np.max(raw_data[:,0])
n_movies = np.max(raw_data[:,1])
shape = (n_users+1,n_movies+1)
shape

(943, 1682)

In [5]:
adj_matrix = np.ndarray(shape,dtype=int)
for user_id, movie_id, rating, time in raw_data:
    adj_matrix[user_id][movie_id] = rating

In [6]:
adj_matrix

array([[5, 3, 4, ..., 0, 0, 0],
       [4, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [5, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 5, 0, ..., 0, 0, 0]])

In [7]:
adj_matrix.shape

(943, 1682)

In [18]:
# U: 사용자, S: 특이 값 ,V: 항목
U, S, V = randomized_svd(adj_matrix, n_components=2)
S = np.diag(S)

In [19]:
print(U.shape)
print(S.shape)
print(V.shape)

(943, 2)
(2, 2)
(2, 1682)


In [20]:
np.matmul(np.matmul(U,S),V)

array([[ 3.91732663e+00,  1.47276644e+00,  7.98261988e-01, ...,
         6.24907189e-04,  1.41100852e-02,  1.36545878e-02],
       [ 1.85777226e+00,  3.96191175e-01,  5.05705740e-01, ...,
         5.38862978e-03,  1.77237914e-03,  5.26968095e-04],
       [ 8.94989517e-01,  1.71578497e-01,  2.51738682e-01, ...,
         2.92094923e-03,  5.39937171e-04, -1.25733753e-04],
       ...,
       [ 9.92051955e-01,  2.10814957e-01,  2.70363365e-01, ...,
         2.89019297e-03,  9.34221962e-04,  2.66612193e-04],
       [ 1.30425401e+00,  5.27669941e-01,  2.50080165e-01, ...,
        -4.20677765e-04,  5.30525683e-03,  5.28069948e-03],
       [ 2.82999397e+00,  9.70812247e-01,  6.15871694e-01, ...,
         2.02091492e-03,  8.67740813e-03,  8.03107892e-03]])

In [ ]:
# 사용자 기반 추천
# 나와 비슷한 취향을 가진 다른 사용자의 행동을 추천
# 사용자 특징 백터의 유사도 사용

In [22]:
def compute_cos_similarity(v1,v2):
    norm_1 = np.sqrt(np.sum(np.square(v1)))
    norm_2 = np.sqrt(np.sum(np.square(v2)))
    dot = np.dot(v1,v2)
    return dot / (norm_1 * norm_2)

In [24]:
my_id, my_vector = 0, U[0]
best_match, best_match_id, best_match_vector = -1,-1, []

for user_id,user_vector in enumerate(U):
    if my_id != user_id:
        # Cos_Similarity
        cos_sim = compute_cos_similarity(my_vector, user_vector)
        if cos_sim > best_match:
            best_match = cos_sim
            best_match_id = user_id
            best_match_vector = user_vector

print('Best Match: {},Best_Match_ID: {}'.format(best_match, best_match_id))

Best Match: 0.9999942295956324,Best_Match_ID: 235


In [26]:
recommend_list = []
for i, log in enumerate(zip(adj_matrix[my_id], adj_matrix[best_match_id])):
    log1, log2 = log
    if log1 < 1. and log2 > 0.:
        recommend_list.append(i)
        
# 내가 보지 않고 235(best match id)이 본 영화 추천
print(recommend_list)

[272, 273, 274, 281, 285, 288, 293, 297, 303, 306, 312, 317, 327, 332, 369, 410, 418, 419, 422, 426, 428, 431, 434, 442, 461, 475, 477, 482, 495, 503, 504, 505, 506, 509, 519, 520, 522, 525, 531, 545, 548, 590, 594, 595, 613, 631, 654, 658, 660, 672, 684, 685, 691, 695, 698, 704, 716, 728, 734, 749, 755, 863, 865, 933, 1012, 1038, 1101, 1327, 1400]


In [ ]:
# 항목 기반 추천
# 내가 본 항목과 비슷한 항목을 추천
# 항목 특징 벡터의 유사도 사용

In [29]:
my_id, my_vector = 0, V.T[0]
best_match, best_match_id, best_match_vector = -1,-1, []

for user_id,user_vector in enumerate(V.T):
    if my_id != user_id:
        # Cos_Similarity
        cos_sim = compute_cos_similarity(my_vector, user_vector)
        if cos_sim > best_match:
            best_match = cos_sim
            best_match_id = user_id
            best_match_vector = user_vector

print('Best Match: {},Best_Match_ID: {}'.format(best_match, best_match_id))

Best Match: 0.9999999951364144,Best_Match_ID: 1287


In [30]:
recommend_list = []
for i, user_vector in enumerate(adj_matrix):
    if adj_matrix[i][my_id] > 0.9:
        recommend_list.append(i)
        
# 내가 보지 않고 1287(best match id)이 본 영화 추천
print(recommend_list)

[0, 1, 4, 5, 9, 12, 14, 15, 16, 17, 19, 20, 22, 24, 25, 37, 40, 41, 42, 43, 44, 48, 53, 55, 56, 57, 58, 61, 62, 63, 64, 65, 66, 69, 71, 72, 74, 76, 78, 80, 81, 82, 83, 88, 91, 92, 93, 94, 95, 96, 98, 100, 101, 105, 107, 108, 116, 119, 120, 123, 124, 127, 129, 130, 133, 136, 137, 140, 143, 144, 147, 149, 150, 156, 157, 159, 161, 167, 173, 176, 177, 180, 181, 183, 188, 192, 193, 197, 198, 199, 200, 201, 202, 203, 208, 209, 212, 215, 221, 222, 229, 230, 231, 233, 234, 241, 242, 243, 245, 246, 247, 248, 249, 250, 251, 252, 253, 255, 261, 262, 264, 267, 270, 273, 274, 275, 276, 278, 279, 285, 286, 288, 289, 290, 291, 292, 293, 294, 295, 296, 297, 298, 300, 302, 304, 306, 307, 310, 311, 312, 313, 319, 321, 323, 324, 325, 326, 329, 330, 331, 335, 337, 338, 339, 342, 343, 344, 346, 347, 349, 356, 358, 359, 362, 364, 370, 373, 377, 378, 379, 380, 386, 387, 388, 389, 392, 393, 394, 395, 397, 398, 400, 401, 402, 405, 406, 410, 411, 415, 416, 418, 421, 423, 424, 428, 431, 433, 434, 437, 440, 444, 